In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\jesus\AppData\Local\Temp\ipykernel_3852\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [12]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv(r"C:\Users\jesus\lab-natural-language-processing\data\kg_train.csv", encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [15]:
from sklearn.model_selection import train_test_split

# Divide the data into training and testing sets
data_train, data_val = train_test_split(data, test_size=0.2, random_state=42)
print(data_train.shape, data_val.shape)

(800, 2) (200, 2)


## Data Preprocessing

In [20]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jesus\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [22]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [31]:
from bs4 import BeautifulSoup
import re

def clean_html(text):
    # Remove inline JavaScript/CSS
    text = re.sub(r'<script.*?>.*?</script>', '', text, flags=re.DOTALL)
    text = re.sub(r'<style.*?>.*?</style>', '', text, flags=re.DOTALL)
    # Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    
    # Ensure the text is not being interpreted as a filename by BeautifulSoup
    if isinstance(text, str):  # Ensure text is a string
        text = BeautifulSoup(text, "html.parser").get_text()
    return text

data['cleaned_text'] = data['text'].apply(clean_html)


C:\Users\jesus\AppData\Local\Temp\ipykernel_3852\4005998394.py:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()
C:\Users\jesus\AppData\Local\Temp\ipykernel_3852\4005998394.py:13: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [34]:
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters and numbers
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)  # Remove single characters
    text = re.sub(r'^\s*[a-zA-Z]\s+', '', text)  # Remove single characters at the start
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    return text.lower()  # Convert to lowercase

data['preprocessed_text'] = data['cleaned_text'].apply(preprocess_text)

## Now let's work on removing stopwords
Remove the stopwords.

In [37]:
stop_words = set(stopwords.words("english"))

def remove_stopwords(text):
    return " ".join([word for word in text.split() if word not in stop_words])

data['preprocessed_text'] = data['preprocessed_text'].apply(remove_stopwords)

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [46]:
import nltk

# Download the 'punkt' tokenizer and the 'punkt_tab' resource
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jesus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\jesus\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [50]:
import nltk

# Download the 'wordnet' corpus
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jesus\AppData\Roaming\nltk_data...


True

In [52]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    words = word_tokenize(text)  # Tokenize the text into words
    return " ".join([lemmatizer.lemmatize(word) for word in words])

# Apply lemmatization to the preprocessed_text column
data['preprocessed_text'] = data['preprocessed_text'].apply(lemmatize_text)


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [55]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['preprocessed_text'])

# Top 10 words in ham and spam
print(vectorizer.get_feature_names_out()[:10])

['aac' 'aaclocated' 'aae' 'aag' 'aaronovitchon' 'abacha' 'abachabefore'
 'abachac' 'abachace' 'abachaco']


## Extra features

In [60]:
print(data_train.columns)  # Check columns of the training set
print(data_val.columns)    # Check columns of the validation set


Index(['text', 'label'], dtype='object')
Index(['text', 'label'], dtype='object')


In [62]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Example function to preprocess the text
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Tokenize the text
    words = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    # Join back the words into a single string
    return ' '.join(words)

# Apply preprocessing to both training and validation sets
data_train['preprocessed_text'] = data_train['text'].apply(preprocess_text)
data_val['preprocessed_text'] = data_val['text'].apply(preprocess_text)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jesus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jesus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [64]:
# Define money symbols and suspicious words
money_symbol_list = "|".join(["euro", "dollar", "pound", "€", "$"])
suspicious_words = "|".join(["free", "cheap", "sex", "money", "account", "bank", "fund", "transfer", "transaction", "win", "deposit", "password"])

# Add the new columns to data_train and data_val
data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_symbol_list) * 1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words) * 1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x))

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_symbol_list) * 1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words) * 1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x))

# Display the updated training data
print(data_train.head())

                                                  text  label  \
29   ----------- REGARDS, MR NELSON SMITH.KINDLY RE...      1   
535  I have not been able to reach oscar this am. W...      0   
695  ; Huma Abedin B6I'm checking with Pat on the 5...      0   
557  I can have it announced here on Monday - can't...      0   
836      BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...      1   

                                     preprocessed_text  money_mark  \
29   regards mr nelson smithkindly reply private em...           1   
535       able reach oscar supposed send pdb u receive           1   
695  huma abedin bim checking pat k work jack jake ...           1   
557                        announced monday cant today           1   
836  bank africaagence san pedro bp san pedro cote ...           1   

     suspicious_words  text_len  
29                  0        76  
535                 0        44  
695                 0        81  
557                 0        27  
836               

## How would work the Bag of Words with Count Vectorizer concept?

In [67]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(data_train['preprocessed_text'])
X_val = vectorizer.transform(data_val['preprocessed_text'])

print(X_train.shape, X_val.shape)

(800, 18221) (200, 18221)


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [70]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(data_train['preprocessed_text'])
X_val_tfidf = vectorizer.transform(data_val['preprocessed_text'])

print(X_train_tfidf.shape, X_val_tfidf.shape)

(800, 18221) (200, 18221)


## And the Train a Classifier?

In [73]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

clf = MultinomialNB()
clf.fit(X_train_tfidf, data_train['label'])

y_pred = clf.predict(X_val_tfidf)
print(f"Accuracy: {accuracy_score(data_val['label'], y_pred):.2f}")

Accuracy: 0.94


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code